In [1]:
# !pip install pandas
# !pip install numpy

In [2]:
import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None # Fuck warnings

<h1> Загрузка данных

In [3]:
import json


with open(r'C:\Users\User\Documents\snli_1.0/snli_1.0_train.jsonl', 'r') as json_file:
    json_list = list(json_file)

result = []
for json_str in json_list:
    result.append(json.loads(json_str))

In [45]:
df_train = pd.json_normalize(result)

In [5]:
df_train = df_train[:10000] # Обрежем данные для тренировки работы с ними

In [6]:
df_train.columns

Index(['annotator_labels', 'captionID', 'gold_label', 'pairID', 'sentence1',
       'sentence1_binary_parse', 'sentence1_parse', 'sentence2',
       'sentence2_binary_parse', 'sentence2_parse'],
      dtype='object')

In [7]:
df_train.head(1)

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...


<h1> Предобработка данных

<h3> Оставляем только основу от слов

In [8]:
import nltk.stem


my_stem = nltk.stem.SnowballStemmer('english')

def stemSentence(sentence):
    new_sentence = ''

    for word in sentence.split():
        new_sentence += my_stem.stem(word) + ' '
    
    return new_sentence[:-1]

<h3> Убираем символы

In [9]:
import string

def deleteSymbs(sentence):
    res = ''

    for char in sentence:
        if char not in string.punctuation:
            res += char
    
    return res

<h3> Agregate function

In [10]:
def fixSentence(sentence, funcs):   # funcs должно включать вышеописанные функции
    for func in funcs:
        sentence = eval(func)(sentence)
    
    return sentence

<h1> Word2Vec

In [11]:
texts = np.concatenate((df_train.Fixed_sentence1.values, df_train.Fixed_sentence2.values))

AttributeError: 'DataFrame' object has no attribute 'Fixed_sentence1'

In [12]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api


# corpus = api.load('text8')  # download the corpus and return it opened as an iterable
# model = Word2Vec(corpus)    # train a model from the corpus
# model.most_similar("car")

In [13]:
# model = api.load('glove-twitter-200') # 758MB
model = api.load('glove-wiki-gigaword-100') # 128MB - гораздо адекватнее

In [14]:
model.most_similar('horse')

[('horses', 0.8550404906272888),
 ('riding', 0.7131650447845459),
 ('dog', 0.7109653353691101),
 ('racing', 0.6629325151443481),
 ('bike', 0.6614626049995422),
 ('thoroughbred', 0.6600428819656372),
 ('bull', 0.6456228494644165),
 ('jockey', 0.6354342699050903),
 ('breeders', 0.6333900690078735),
 ('race', 0.6270371675491333)]

<h3> Подготовка данных для w2v

In [15]:
for i in [1, 2]:
    df_train[f'Fixed_sentence{i}'] = df_train[f'sentence{i}'].apply(lambda sentence: fixSentence(sentence, ['deleteSymbs']))

<h3> Word2Vec + Logistic regression

In [16]:
def getSentenceVecSum(sentence, model):
    result_vec = np.zeros(100)
    for word in sentence.split():
        try:
            result_vec += model.wv[word]
        except:
            pass
    
    return result_vec

In [17]:
def getDistanceBetweenVecs(vec1, vec2):
    return np.linalg.norm(vec1-vec2)

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


sentence1 = df_train.Fixed_sentence1.apply(lambda sentence: getSentenceVecSum(sentence, model))
sentence2 = df_train.Fixed_sentence2.apply(lambda sentence: getSentenceVecSum(sentence, model))
x = sentence1 - sentence2

new_x = []
for i in range(len(x)):
    new_x.append([])
    for j in range(len(x[0])):
        new_x[i].append(x[i][j])

x = pd.DataFrame(new_x)

y = df_train.gold_label.apply(lambda label: {'entailment': 0, 'contradiction': 1, 'neutral': 2, '-': 3}[label])

random_seed = 12
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=random_seed)

logRegr = LogisticRegression()
logRegr = logRegr.fit(x_train, y_train)

cv_results = cross_val_score(logRegr, x_train, y_train, cv=5, scoring='accuracy')

print('_'*80, cv_results, '_'*80, sep='\n')

<ipython-input-16-0c51de18855c>:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  result_vec += model.wv[word]
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.h

10000